# Data Collection and Pre-Processing - Lab

**Name:** Jarius Bedward

**Course:** PROG8245 Machine Learning Programming

**Date:** 2025-09-30

### Imports & Setup

In [13]:
import pandas as pd
import numpy as np
from dataclasses import dataclass, field



# Paths for primary and secondary data

sales_df = pd.read_csv("data/1000 Sales Records.csv")
catalog_df = pd.read_csv("data/shop-product-catalog.csv")




### Step 1: Hello, Data!

Load the primary transactions file and display the first 3 rows

In [14]:
display(sales_df.head(3))
display(catalog_df.head(3))

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


,ProductID,ProductName,ProductBrand,Gender,Price,Description,PrimaryColor
0,1000,Presto Fly 3,Nike,Women,10999,Lightweight and breathable running shoes with ...,Pink
1,1001,Superstar Bold 2,Adidas,Men,11999,Classic basketball shoes with a modern twist a...,Black
2,1002,Old Skool Lite 3,Vans,Unisex,4999,Iconic skateboarding shoes with a relaxed fit ...,Grey


### Step 2: Pick the right container

For this dataset we would use a dictionary of lists from padnas dataframe.
Pandas dataframe is built on dictionary like structures so its the most appropriate for this

### Step 3: Order Class

In [15]:

class SalesCleaner:
    def __init__(self, df):
        self.df = df.copy()

    def clean(self):
        #Cleaning: Strip the whitespace in item type
        self.df["Item Type"] = self.df["Item Type"].str.strip()
        return self.df

    def total_revenue(self):

        return self.df ["Total Revenue"].sum()

cleaner = SalesCleaner(sales_df)
cleaned_sales = cleaner.clean()
print("Total Revenue:", cleaner.total_revenue())

Total Revenue: 1327321840.33


### Step 4: Bulk Load Orders

In [16]:
profit_map = dict(zip(cleaned_sales["Order ID"], cleaned_sales["Total Profit"]))
list(profit_map.items())[:5]

[(686800706, 1468506.02),
 (185941302, 190526.34),
 (246222341, 145419.62),
 (161442649, 294295.98),
 (645713555, 23726.45)]

### Step 5: Quick Profiling

In [17]:
print("Min price:", cleaned_sales["Unit Price"].min())
print("Mean price:", cleaned_sales["Unit Price"].mean())
print("Max price:", cleaned_sales["Unit Price"].max())
print("Unique countries:", len(set(cleaned_sales["Country"])))

Min price: 9.33
Mean price: 262.10684
Max price: 668.27
Unique countries: 185


### Step 6: Spot the Grime

Identify 3 dirty cases

    1. Extra whitespace in "Item type"
    2. Inconsistent date formats in "order date
    3. Negative or zero "Units sold"

In [18]:
print("Whitespace check:", cleaned_sales["Item Type"].str.contains(" ").sum())
print("date format sample:", cleaned_sales["Order Date"].head())
print("Units sold ≤ 0:", (cleaned_sales["Units Sold"] <= 0).sum())

Whitespace check: 263
date format sample: 0    10/18/2014
1     11/7/2011
2    10/31/2016
3     4/10/2010
4     8/16/2011
Name: Order Date, dtype: object
Units sold ≤ 0: 0


### Step 7: Clean Rules

We apply rules inside the clean method to normalize thew whitespace, convert dates to datetime and filter out bad units

In [19]:
def clean_sales(df):
    df = df.copy()
    df["Item Type"] = df["Item Type"].str.strip()
    df["Order Date"] = pd.to_datetime(df["Order Date"])
    df["Ship Date"] = pd.to_datetime(df["Ship Date"])
    df = df[df["Units Sold"] > 0]
    return df

sales_cleaned = clean_sales(sales_df)
sales_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Region          1000 non-null   object        
 1   Country         1000 non-null   object        
 2   Item Type       1000 non-null   object        
 3   Sales Channel   1000 non-null   object        
 4   Order Priority  1000 non-null   object        
 5   Order Date      1000 non-null   datetime64[ns]
 6   Order ID        1000 non-null   int64         
 7   Ship Date       1000 non-null   datetime64[ns]
 8   Units Sold      1000 non-null   int64         
 9   Unit Price      1000 non-null   float64       
 10  Unit Cost       1000 non-null   float64       
 11  Total Revenue   1000 non-null   float64       
 12  Total Cost      1000 non-null   float64       
 13  Total Profit    1000 non-null   float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(5)
mem

### Step 8: Transformations

In [20]:
priority_map = {"H": 1, "M": 2, "L": 3, "C":4}
sales_cleaned["Priority_code"] = sales_cleaned["Order Priority"].map(priority_map)
sales_cleaned[["Order Priority", "Priority_code"]].head()

,Order Priority,Priority_code
0,M,2
1,M,2
2,C,4
3,C,4
4,H,1


### Step 9: Feature Engineering

In [21]:

sales_cleaned["Days_to_Ship"] = (sales_cleaned["Ship Date"] - sales_cleaned["Order Date"]).dt.days
sales_cleaned[["Order Date", "Ship Date", "Days_to_Ship"]].head()

,Order Date,Ship Date,Days_to_Ship
0,2014-10-18,2014-10-31,13
1,2011-11-07,2011-12-08,31
2,2016-10-31,2016-12-09,39
3,2010-04-10,2010-05-12,32
4,2011-08-16,2011-08-31,15


### Step 10: Mini-Aggregation

In [22]:
revenue_by_country = sales_cleaned.groupby("Country")["Total Revenue"].sum().sort_values(ascending=False)
revenue_by_country.head(10)

Country
Cuba                        27522085.87
Papua New Guinea            22209652.47
Ghana                       21267908.50
Costa Rica                  19628279.63
Iran                        18719532.99
Chad                        17278040.69
Central African Republic    16591036.85
Malaysia                    16416989.18
Austria                     16199378.41
Vanuatu                     16011801.84
Name: Total Revenue, dtype: float64

### Step 11: Serialization

In [23]:
sales_cleaned.to_csv("data/sales_cleaned.csv", index=False)
sales_cleaned.to_json("data/sales_cleaned.json", orient="records", lines=True)

### Step 12: Soft Interview Reflection

Functions as well as our class allowed us to structure the workflow into clear adn reusable blocks.
Instead of repeating code for cleaning and aggregation, we called methods. This reduces the errors

### Data-Dictionary Section

## 📑 Data Dictionary

| Field           | Type      | Description                                       | Source   | Notes on Creation |
|-----------------|----------|---------------------------------------------------|----------|-------------------|
| Region          | String   | Geographic sales region (e.g., Asia, Europe)      | Sales    | Original field from sales dataset |
| Country         | String   | Country of transaction                            | Sales    | Original field from sales dataset |
| Item Type       | String   | Product category (e.g., Fruits, Baby Food)        | Sales    | Original field from sales dataset |
| Sales Channel   | String   | Online or Offline sales channel                   | Sales    | Original field from sales dataset |
| Order Priority  | String   | Priority code (H = High, M = Medium, L = Low, C = Critical) | Sales | Original field; also transformed into numeric codes |
| Order Date      | Date     | Date when the order was placed                    | Sales    | Original, converted to `datetime` |
| Order ID        | Integer  | Unique identifier for the sales order             | Sales    | Original field |
| Ship Date       | Date     | Date when the order shipped                       | Sales    | Original, converted to `datetime` |
| Units Sold      | Integer  | Quantity of items sold                            | Sales    | Original field |
| Unit Price      | Float    | Price per unit (USD)                              | Sales    | Original field |
| Unit Cost       | Float    | Cost per unit (USD)                               | Sales    | Original field |
| Total Revenue   | Float    | Total revenue = Units Sold × Unit Price           | Sales    | Derived field (calculated in dataset) |
| Total Cost      | Float    | Total cost = Units Sold × Unit Cost               | Sales    | Derived field (calculated in dataset) |
| Total Profit    | Float    | Profit = Revenue − Cost                           | Sales    | Derived field (calculated in dataset) |
| Priority_Code   | Integer  | Encoded priority (H=1, M=2, L=3, C=4)              | Synthetic| Added during transformation step |
| Days_to_Ship    | Integer  | Days between order and shipment                   | Synthetic| Derived feature using Order Date and Ship Date |
| ProductID       | Integer  | Unique identifier for catalog product             | Catalog  | Original field from product catalog |
| ProductName     | String   | Official product name (e.g., Presto Fly 3)        | Catalog  | Original field |
| ProductBrand    | String   | Brand name (Nike, Adidas, Vans, etc.)             | Catalog  | Original field |
| Gender          | String   | Intended gender for product (Men, Women, Unisex)  | Catalog  | Original field |
| Price           | Integer  | Catalog price in cents (e.g., 10999 = $109.99)    | Catalog  | Original field |
| Description     | String   | Marketing description of the product              | Catalog  | Original field |
| PrimaryColor    | String   | Dominant color of the product                     | Catalog  | Original field |

---

### Explanation:
- **Original fields**: Taken from the dataset directly (e.g, Country, ProductName)
- **Created Fields**: Calculated using existing values (e.g, Total profit = Revenue - Cost)
- **Synthetic Fields**: Created during this lab for analysis (e.g, Days_to_Ship)
- **Combination fields**: None but it could be created (e.g, merging Item Type with ProductBrand)

In [ ]:
##